In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_backup = pd.read_csv('../csvs/deliveries.csv')

In [3]:
data_backup.head(5)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.1,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.2,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.3,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.4,England,New Zealand,DJ Malan,JM Bairstow,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2023/24,2023-10-05,"Narendra Modi Stadium, Ahmedabad",1,0.5,England,New Zealand,JM Bairstow,DJ Malan,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data = data_backup.copy()
data.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [5]:
original_df = data

original_df['start_date'] = pd.to_datetime(original_df['start_date'])


grouped_df = original_df.groupby(['match_id', 'innings', 'venue'])


result_df = grouped_df.agg({
    'batting_team': 'first',
    'bowling_team': 'first',
    'wides': 'sum',
    'noballs': 'sum',
    'runs_off_bat': 'sum',
    'ball': 'max',
    'wicket_type': lambda x: x.notnull().sum(),
    'start_date': 'first'
}).reset_index()


result_df.columns = ['match_id', 'innings', 'venue', 'batting_team', 'bowling_team',
                      'total_wides', 'total_noballs', 'total_runs_per_innings_match',
                      'last_ball', 'total_wickets', 'start_date']


result_df['total_overs_played'] = result_df['last_ball'].apply(lambda x: min((int(x) + round((x % 1) * 10, 4) / 6), 50.0))


result_df[['total_wides', 'total_noballs']] = result_df[['total_wides', 'total_noballs']].astype(int)


result_df['total_runs_per_innings_match'] += result_df['total_wides'] + result_df['total_noballs']

In [6]:
result_df.head()

,match_id,innings,venue,batting_team,bowling_team,total_wides,total_noballs,total_runs_per_innings_match,last_ball,total_wickets,start_date,total_overs_played
0,1,1,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,6,0,282,49.7,9,2023-10-05,50.000000
1,1,2,"Narendra Modi Stadium, Ahmedabad",New Zealand,England,3,0,278,36.2,1,2023-10-05,36.333333
2,2,1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,8,1,286,48.6,10,2023-10-06,49.000000
3,2,2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Netherlands,Pakistan,9,0,205,40.7,10,2023-10-06,41.166667
4,3,1,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,8,0,156,37.2,10,2023-10-07,37.333333


In [7]:
result_df = result_df[result_df['innings'] == 1]

In [8]:
result_df.head()

,match_id,innings,venue,batting_team,bowling_team,total_wides,total_noballs,total_runs_per_innings_match,last_ball,total_wickets,start_date,total_overs_played
0,1,1,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,6,0,282,49.7,9,2023-10-05,50.000000
2,2,1,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,8,1,286,48.6,10,2023-10-06,49.000000
4,3,1,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,8,0,156,37.2,10,2023-10-07,37.333333
6,4,1,"Arun Jaitley Stadium, Delhi",South Africa,Sri Lanka,21,1,427,49.7,5,2023-10-07,50.000000
8,5,1,"MA Chidambaram Stadium, Chepauk, Chennai",Australia,India,6,0,193,49.3,10,2023-10-08,49.500000


In [9]:
result_df.drop(['match_id','innings','total_wides','total_noballs','last_ball','total_wickets','start_date'],axis=1,inplace=True)

In [10]:
result_df.head()

,venue,batting_team,bowling_team,total_runs_per_innings_match,total_overs_played
0,"Narendra Modi Stadium, Ahmedabad",England,New Zealand,282,50.000000
2,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Pakistan,Netherlands,286,49.000000
4,"Himachal Pradesh Cricket Association Stadium, ...",Afghanistan,Bangladesh,156,37.333333
6,"Arun Jaitley Stadium, Delhi",South Africa,Sri Lanka,427,50.000000
8,"MA Chidambaram Stadium, Chepauk, Chennai",Australia,India,193,49.500000


In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [12]:
mapping = {}
categorical_columns = ['venue','batting_team','bowling_team']

for column in categorical_columns:
    result_df[column] = le.fit_transform(result_df[column])
    mapping[column] = dict(zip(le.classes_, le.transform(le.classes_)))

In [13]:
result_df.head()

,venue,batting_team,bowling_team,total_runs_per_innings_match,total_overs_played
0,7,3,6,282,50.000000
2,8,7,5,286,49.000000
4,3,0,2,156,37.333333
6,0,8,9,427,50.000000
8,5,1,4,193,49.500000


In [14]:
X=result_df.drop('total_runs_per_innings_match',axis='columns')
y=result_df['total_runs_per_innings_match']

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=42)

size_scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = size_scaler.transform(X_train)
X_test_scaled = size_scaler.transform(X_test)
X_train_scaled.shape, X_test_scaled.shape

((28, 4), (4, 4))

In [16]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train,y_train)

model.score(X_test,y_test)

0.5044757399585037

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler


model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer with 1 unit and linear activation
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(X_train, y_train, epochs=500, batch_size=1, verbose=1, callbacks=[early_stopping], validation_split=0.2)

Epoch 1/500
22/22 [==============================] - 1s 8ms/step - loss: 85438.7969 - mae: 283.3488 - val_loss: 93010.6562 - val_mae: 297.4750
Epoch 2/500
22/22 [==============================] - 0s 2ms/step - loss: 81127.3125 - mae: 275.7263 - val_loss: 89741.3359 - val_mae: 291.9631
Epoch 3/500
22/22 [==============================] - 0s 3ms/step - loss: 77762.5625 - mae: 269.4910 - val_loss: 85148.0625 - val_mae: 284.0350
Epoch 4/500
22/22 [==============================] - 0s 2ms/step - loss: 72673.8359 - mae: 260.1079 - val_loss: 78794.6484 - val_mae: 272.6593
Epoch 5/500
22/22 [==============================] - 0s 3ms/step - loss: 65678.0859 - mae: 246.8274 - val_loss: 70135.6406 - val_mae: 256.4094
Epoch 6/500
22/22 [==============================] - 0s 2ms/step - loss: 56417.3594 - mae: 227.2188 - val_loss: 58097.5898 - val_mae: 231.9599
Epoch 7/500
22/22 [==============================] - 0s 2ms/step - loss: 44390.6055 - mae: 198.6962 - val_loss: 42970.0898 - val_mae: 197.0166

In [18]:
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
print(f"Training MAE: {train_mae}")

# Evaluate on the test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test MAE: {test_mae}")

1/1 [==============================] - 0s 110ms/step - loss: 3760.1853 - mae: 49.4651
Training MAE: 49.465126037597656
1/1 [==============================] - 0s 19ms/step - loss: 2441.4155 - mae: 43.2130
Test MAE: 43.21296310424805


In [19]:
y_pred = model.predict(X_test)
y_pred

1/1 [==============================] - 0s 61ms/step


array([[283.132  ],
       [278.93796],
       [206.26212],
       [285.60434]], dtype=float32)

In [20]:
y_test

58    239
30    284
48    153
34    356
Name: total_runs_per_innings_match, dtype: int64

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestRegressor()

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
max_features = ['auto','sqrt']
max_depth = [int(x) for x in np.linspace(5,30,num=6)]
min_samples_split = [2,5,10,15,100]
min_samples_leaf = [1,2,5,10]

random_grid = {'n_estimators':n_estimators,
                'max_features':max_features,
                'max_depth':max_depth,
                'min_samples_split':min_samples_split,
                'min_samples_leaf':min_samples_leaf}

rf_random = RandomizedSearchCV(estimator=rf,param_distributions=random_grid,scoring='neg_mean_squared_error',n_iter=10,cv=5,verbose=2,random_state=42,n_jobs=1)

rf_random.fit(X_train,y_train)

print(rf_random.best_params_)

print(rf_random.best_score_)

predictions = rf_random.predict(X_test)
print(predictions)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=5, min_samples_split=5, n_estimators=900; total time=   0.4s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=   0.5s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=1100; total time=   0.5s
[CV] END max_depth=15, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimator

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 638,

{'n_estimators': 1100, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 15}
-4019.6655673604955
[311.05258614 275.53650612 231.51481852 299.10836607]


In [22]:
y_test

58    239
30    284
48    153
34    356
Name: total_runs_per_innings_match, dtype: int64

In [23]:
y_pred = rf_random.predict(X_test)
print(abs(y_pred-y_test))

58    72.052586
30     8.463494
48    78.514819
34    56.891634
Name: total_runs_per_innings_match, dtype: float64


In [24]:
print(np.mean(abs(y_pred-y_test)))

print(np.mean(abs(rf_random.predict(X_train)-y_train)))

53.9806331204701
36.024361691824005


In [25]:
import pickle as pkl

In [26]:
pkl.dump(model, open('../pickles/inning1_run_predictor', 'wb'))